# Scraping the Unscrapable

### What happens if I try to parse Google with `requests` and `BeautifulSoup`?

In [ ]:
import requests
from bs4 import BeautifulSoup

google_url="https://www.google.com"
soup=BeautifulSoup(requests.get(google_url).text, "html5lib")
print(soup.prettify())

There is a lot going on on this page - specifically, JavaScript is being used to render a lot of stuff here. 

Nevertheless, we can use Chrome inspector to find the name of a particular element. On Mac, the shortcut is "Alt/Option + Command + I". Then, use "Command + Shift + C" to bring up the "hover highlight" functionality: 

In [2]:
print(soup.find(title="Google Search"))

<input autocomplete="off" class="lst" maxlength="2048" name="q" size="57" style="color:#000;margin:0;padding:5px 8px 0 6px;vertical-align:top" title="Google Search" value=""/>


That's great, but how can I actually type something in that box using Python? For that, we need Selenium.

## Installing Selenium

* Go to [this website](https://sites.google.com/a/chromium.org/chromedriver/home) to download Selenium Mac driver.
* Use the command line or click on the file name in your Downloads folder to unzip the file.
* Use the `mv` command to move the new `chromedriver` executable file into your home directory.

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

### CHANGE THE NAME "seth" HERE:
chromedriver_path = "/Users/seth/chromedriver"

driver = webdriver.Chrome(chromedriver_path)
driver.get('http://www.google.com/')
time.sleep(1); 
search_box = driver.find_element_by_name("q")

Note that running this code will actually open a new Chrome window!

The standard function that we use to "send data" to `WebElements` in Selenium is called `send_keys`.

In [4]:
search_box.send_keys("What to wear today")
search_box.send_keys(Keys.RETURN)

# OpenTable

Now let's go through an example of interacting with a dynamic website using Selenium.

In [5]:
import time
driver = webdriver.Chrome(chromedriver_path)
driver.get('http://www.opentable.com/')
time.sleep(1); 

This selects "3 people" from the first form:

In [6]:
people_dropdown = driver.find_element_by_name('Select_1')
time.sleep(1); 
people_dropdown.send_keys("3 person")
people_dropdown.send_keys(Keys.RETURN)
time.sleep(1); 

This selects Friday as the date:

In [7]:
temp = driver.find_element_by_name('datepicker')
time.sleep(1); 
temp.click()
time.sleep(1); 
date_element = driver.find_element_by_xpath('//div[@data-pick="1500008400000"]')
date_element.click()

This selects 8:00 PM as the time:

In [8]:
time_dropdown = driver.find_element_by_name('Select_0')
time_dropdown.send_keys("8:00 PM")
time_dropdown.send_keys(Keys.RETURN)
time.sleep(1); 

This types in the "West Loop" as the neighborhood:

In [14]:
location = driver.find_element_by_name('searchText')
location.send_keys("West Loop")
location.send_keys(Keys.RETURN)
time.sleep(1); 

Now, let's search!

In [15]:
search = driver.find_element_by_xpath('//input[@value="Find a Table"]')
search.click()
time.sleep(1); 

Now, let's loop through all of the restaurants that appear here. Note: at this point, you could switch back and use the slightly simpler and easier-to-use Beautiful Soup.

In [21]:
spans = driver.find_elements_by_xpath('//span[@class="rest-row-name-text"]')
for span in spans[:10]:
    print(span.text)

You would use only slightly more complicated code to, for example:

* Get the number of "dollar signs" for each restaurant and print that out.
* Get a list of restaurants that had reservations available at exactly 8:00 PM.
* Print out the rating, out of five stars, of each restaurant.

Each of these would take a bit of time and experimentation to figure out, but hopefully you see that it is possible.

Let's click on "Bar Siena":

In [17]:
for span in spans:
    if span.text == "Bar Siena":
        span.click()

Uh oh. We opened a new tab, but our driver is still on the old tab, so 
```
driver.find_element_by_xpath('//p[@class="readmore"]')
```
for example, won't work!

We can use the following to switch the driver to the correct window:

In [18]:
driver.switch_to_window(driver.window_handles[1])

In [ ]:
description_element = driver.find_element_by_xpath('//p[@class="readmore"]')
description_text = description_element.text.replace('\n', ' ')
description_text

Finally, let's make a reservation!

In [ ]:
reservation_element = driver.find_element_by_xpath('//a[@data-datetime="2017-07-14 22:00"]')
reservation_element.click()

In [ ]:
el = driver.find_element_by_xpath('//a[@id="ReservationChangeUserLink"]')
el.click()

You would then enter your email and password and log in. Websites make this hard to do programmatically, even with tools like Selenium, on purpose.

Finally, you can click the button below to confirm the reservation!

In [ ]:
final_button = driver.find_element_by_xpath('//button[@id="btn-complete"]')
final_button.click()

# Selenium References

References: 
- Documentation on finding elements:
- http://selenium-python.readthedocs.org/en/latest/locating-elements.html
- Xpath tutorial:
-  http://www.w3schools.com/xpath/xpath_syntax.asp